In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import re
import logging
from typing import Set

PROJECT_ROOT = Path(__file__).parent.parent if '__file__' in locals() else Path('../')
RAW_DATA_DIR = PROJECT_ROOT / "data" / "raw"
PROCESSED_DATA_DIR = PROJECT_ROOT / "data" / "processed"
STOPWORDS_FILE = PROJECT_ROOT / "data" / "vietnamese-stopwords.txt"

PROCESSED_DATA_DIR.mkdir(exist_ok=True, parents=True)

DEFAULT_ENCODING = "utf-8"
RANDOM_STATE = 42

In [49]:
def load_stopwords(stopwords_path: str) -> Set[str]:
    try:
        with open(stopwords_path, 'r', encoding='utf-8') as f:
            stopwords = set(word.strip().lower() for word in f.readlines() if word.strip())
        return stopwords
    except FileNotFoundError:
        print(f"Stopwords file not found: {stopwords_path}")
        return set()
    except Exception as e:
        print(f"Error loading stopwords: {e}")
        return set()

def remove_emojis(text: str) -> str:
    if pd.isna(text):
        return text

    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002500-\U00002BEF"  # chinese char
        "\U00002702-\U000027B0"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "\U0001f926-\U0001f937"
        "\U00010000-\U0010ffff"
        "\u2640-\u2642"
        "\u2600-\u2B55"
        "\u200d"
        "\u23cf"
        "\u23e9"
        "\u231a"
        "\ufe0f"  # dingbats
        "\u3030"
        "]+",
        flags=re.UNICODE
    )

    text = emoji_pattern.sub(r'', text)
    emoticon_pattern = re.compile(
        r'[:;=8][\-o\*\']?[\)\]\(\[dDpP/\:\}\{@\|\\]|[\)\]\(\[dDpP/\:\}\{@\|\\][\-o\*\']?[:;=8]')
    text = emoticon_pattern.sub(r'', text)
    return text.strip()

def remove_stopwords(text: str, stopwords: Set[str]) -> str:
    if pd.isna(text) or not stopwords:
        return text
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stopwords]
    return ' '.join(filtered_words)

def clean_sentiment_text(text: str, stopwords: Set[str]) -> str:
    pattern = r'(.)\1{2,}'
    if pd.isna(text):
        return text
    
    text = remove_emojis(text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'[""''"\']+', '', text)
    text = remove_stopwords(text, stopwords)
    text = re.sub(pattern, r'\1\1', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def clean_summarization_text(text: str) -> str:
    if pd.isna(text):
        return text
    
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'[.]{3,}', '...', text)
    text = re.sub(r'[!]{2,}', '!', text)
    text = re.sub(r'[?]{2,}', '?', text)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

stopwords = load_stopwords(str(STOPWORDS_FILE))

NameError: name 'Set' is not defined

In [ ]:
def preprocess_summarization(sample_fraction=0.5):
    print("Processing summarization dataset...")
    
    file_path = RAW_DATA_DIR / "data_summary.csv"
    df = pd.read_csv(file_path, encoding=DEFAULT_ENCODING)
    df = df[["Text", "Summary"]]
    
    df_clean = df.dropna(subset=["Text", "Summary"])
    print(f"Dropped {len(df) - len(df_clean)} rows with missing values")
    
    if sample_fraction < 1.0:
        df_clean = df_clean.sample(frac=sample_fraction, random_state=RANDOM_STATE).reset_index(drop=True)
        print(f"Sampled {sample_fraction:.1%} of data")
    
    print("Cleaning Text and Summary columns...")
    df_clean['Text'] = df_clean['Text'].apply(clean_summarization_text)
    df_clean['Summary'] = df_clean['Summary'].apply(clean_summarization_text)
    
    initial_count = len(df_clean)
    df_processed = df_clean[
        (df_clean['Text'].str.strip() != '') &
        (df_clean['Summary'].str.strip() != '')
    ].reset_index(drop=True)
    
    removed_count = initial_count - len(df_processed)
    if removed_count > 0:
        print(f"Removed {removed_count} rows with empty text after cleaning")
    
    output_file = PROCESSED_DATA_DIR / "summary_clean.csv"
    df_processed.to_csv(output_file, index=False, encoding=DEFAULT_ENCODING)
    
    print(f"Final summarization dataset shape: {df_processed.shape}")
    return df_processed

summary_df = preprocess_summarization()

Cảm xúc: 31460 dòng
Tóm tắt: 102681 dòng
Tóm tắt: 102681 dòng


In [ ]:
def preprocess_sentiment(sample_fraction=1.0):
    print("Processing sentiment dataset...")
    
    file_path = RAW_DATA_DIR / "data_sentiment.csv"
    df = pd.read_csv(file_path, encoding=DEFAULT_ENCODING)
    
    df_clean = df.dropna(subset=["comment", "label"])
    print(f"Dropped {len(df) - len(df_clean)} rows with missing values")
    
    if sample_fraction < 1.0:
        df_clean = df_clean.sample(frac=sample_fraction, random_state=RANDOM_STATE).reset_index(drop=True)
        print(f"Sampled {sample_fraction:.1%} of data")
    
    print("Cleaning comment column...")
    df_clean['comment'] = df_clean['comment'].apply(lambda x: clean_sentiment_text(x, stopwords))
    
    initial_count = len(df_clean)
    df_processed = df_clean[df_clean['comment'].str.strip() != ''].reset_index(drop=True)
    df_processed = df_processed[df_processed['comment'].str.len() >= 10].reset_index(drop=True)
    
    removed_count = initial_count - len(df_processed)
    if removed_count > 0:
        print(f"Removed {removed_count} rows with empty/short comments after cleaning")
    
    output_file = PROCESSED_DATA_DIR / "reviews_clean.csv"
    df_processed.to_csv(output_file, index=False, encoding=DEFAULT_ENCODING)
    
    print(f"Final sentiment dataset shape: {df_processed.shape}")
    return df_processed

sentiment_df = preprocess_sentiment()

Test: 'Áo bao đẹp ạ!!' -> 'áo bao đẹp'


In [ ]:
print("Data preprocessing completed successfully!")
print(f"Summarization data: {summary_df.shape[0]} records")
print(f"Sentiment data: {sentiment_df.shape[0]} records")

print(f"\nLabel distribution:")
print(sentiment_df['label'].value_counts())

Dữ liệu phân tích cảm xúc: 22479 records
Dữ liệu tóm tắt: 102393 records
